In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

In [4]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'CMX'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=20)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 0

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [5]:
#Loading SELL'S DATA from a function
data_ventas = data_ventas_query(ciudad=city_code, tipo_cambio=t_cambio, desde=customer_date_desde, hasta = todays_date, tipo_negocio=1)

In [6]:
data_ventas.head()

,submit_date,close_date,month,order_id,order_item_id,tracking_code,customer_id,email_address,name,gmv_usd,discount_applied,cant,unidades,sku_id,padre_sku_id,region_code,category,subcat,adjustment_reason
0,2023-05-05 21:29:16.176,2023-05-12,2023-05,21182747,92438149,290721607_24356307022114,290721607,rrober12591259@gmail.com,Crema Ácida Alpura x 4 lt Bote x 4 L - Unidad,12.41119,0.00000,1.0,UNID,93264,74166,CMX,Lácteos & Huevos,Cremas,None
1,2023-05-05 21:29:16.176,2023-05-12,2023-05,21182747,92438821,290721607_24518289138794,290721607,rrober12591259@gmail.com,Sal Elefante Cocina 1Kg - Bolsa 1kg,0.67175,0.00000,1.0,UNID,-304431,-201554,CMX,Abarrotes & Despensa,Sal & Sazonadores,None
2,2023-05-05 21:29:16.176,2023-05-12,2023-05,21182747,92438819,290721607_24518289138794,290721607,rrober12591259@gmail.com,Detergente en Polvo Roma x 1 kg Bolsa 1kg - Un...,3.33231,0.13333,2.0,UNID,-305196,-201698,CMX,Aseo e Higiene,"Detergente, Jabón & Lavatrastes",Hook_CMX_Detergente_en_Polvo_Roma_x_1_kg_4_%MA...
3,2023-05-05 21:29:16.176,2023-05-12,2023-05,21182747,92438839,290721607_24518289138794,290721607,rrober12591259@gmail.com,Zanahoria jugo Estándar - Kilo,1.27226,0.00000,4.0,KG,560430,403459,CMX,Frutas & Verduras,Verduras,None
4,2023-05-05 21:29:16.176,2023-05-12,2023-05,21182747,92438829,290721607_24518289138794,290721607,rrober12591259@gmail.com,Pasta Espagueti La Moderna x 200 gr Sobre x 2...,3.76590,0.00000,10.0,UNID,136394,-201673,CMX,Abarrotes & Despensa,Pasta,None


In [11]:
info_cat = data_ventas.loc[data_ventas.category == 'Frutas & Verduras',['customer_id','gmv_usd']].groupby('customer_id').gmv_usd.mean().reset_index()

In [12]:
from sklearn.model_selection import train_test_split 

customers = info_cat.customer_id.unique()
print("Total length: ", len(customers))
customer_control, customer_test = train_test_split(customers, test_size = 0.50)

alpha=0.01
aux=0
while abs(1-(info_cat[info_cat.customer_id.isin(customer_control)]['gmv_usd'].mean()/info_cat[info_cat.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:
    customer_control, customer_test = train_test_split(customers, test_size = 0.50)
    aux+=1
    if aux == 1000:
        alpha+=0.01
        aux=0
        
control = customer_control.tolist()
test = customer_test.tolist()

Total length:  5787


In [13]:
global_segment_list = [] #//for procesing
offer_name = []

offer_name.append("HOOKS_EXP_FRUVER_"+todays_date+"_"+city_code)
d = {'name':"HOOKS_EXP_FRUVER_"+todays_date+"_"+city_code, 'customersIds': list(test)}
e = {'name':"HOOKS_EXP_FRUVER_"+todays_date+"_"+city_code+"_CONTROL", 'customersIds': list(control)}

global_segment_list.append(d)
global_segment_list.append(e)

In [14]:
import requests
import json

url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [15]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [16]:
sgtx

[(171005, 'HOOKS_EXP_FRUVER_2023-05-31_CMX'),
 (171006, 'HOOKS_EXP_FRUVER_2023-05-31_CMX_CONTROL')]